In [1]:
import vedo
import napari_shape_odyssey as nso
import napari
import pandas as pd
import numpy as np
from sklearn import decomposition, manifold, cluster
import matplotlib.pyplot as plt

from napari_skimage_regionprops import TableWidget

In [2]:
viewer = napari.Viewer()

In [3]:
mesh = vedo.load(r'C:\Users\johamuel\ownCloud\BiAPoLprojects\113_invading_spheroids\data\Collagen_1\surfaces\surface_simple_frame_24.obj')
mesh = mesh.split()[0].clean().decimate(0.2)
viewer.add_surface((mesh.points(), np.asarray(mesh.faces())), name='Original mesh')

<Surface layer 'Original mesh' at 0x22869892a00>

c:\Users\johamuel\AppData\Local\mambaforge\envs\napari-shape-odyssey\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Napari status bar display of label properties disabled because https://github.com/napari/napari/issues/5417 and https://github.com/napari/napari/issues/4342
Selected eigenvector_1
Selected eigenvector_2
Selected eigenvector_3
Selected eigenvector_0


## Shape spectrum

In [4]:
eigenvectors, eigenvalues = nso.spectral.shape_DNA(viewer.layers[0].data)
feature_table = pd.DataFrame(
    eigenvectors, columns=[f"eigenvector_{i}" for i in range(100)]
)

c:\Users\johamuel\AppData\Local\mambaforge\envs\napari-shape-odyssey\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
viewer.layers[0].features = feature_table

In [6]:
data = list(viewer.layers[0].data)
data[2] = feature_table["eigenvector_30"].values
viewer.layers[0].data = tuple(data)

In [7]:
widget = TableWidget(viewer.layers[0], viewer)
viewer.window.add_dock_widget(widget, area='right')

Napari status bar display of label properties disabled because https://github.com/napari/napari/issues/5417 and https://github.com/napari/napari/issues/4342


In [20]:
viewer.layers[0].features

,eigenvector_0,eigenvector_1,eigenvector_2,eigenvector_3,eigenvector_4,eigenvector_5,eigenvector_6,eigenvector_7,eigenvector_8,eigenvector_9,...,eigenvector_90,eigenvector_91,eigenvector_92,eigenvector_93,eigenvector_94,eigenvector_95,eigenvector_96,eigenvector_97,eigenvector_98,eigenvector_99
0,0.002243,0.001544,0.004842,0.001431,0.009627,-0.003344,-0.003325,-0.009844,-0.001614,-0.001825,...,-0.002000,-0.000144,0.001872,-0.000369,0.001133,0.001282,0.000478,0.000990,0.002509,-0.002171
1,0.002243,0.001544,0.004841,0.001430,0.009622,-0.003342,-0.003322,-0.009836,-0.001612,-0.001823,...,-0.001976,-0.000142,0.001849,-0.000364,0.001119,0.001266,0.000471,0.000978,0.002476,-0.002142
2,0.002243,0.001544,0.004842,0.001431,0.009627,-0.003345,-0.003325,-0.009844,-0.001614,-0.001825,...,-0.002000,-0.000144,0.001872,-0.000369,0.001134,0.001282,0.000477,0.000991,0.002511,-0.002172
3,0.002243,0.001548,0.004817,0.001410,0.009534,-0.003301,-0.003287,-0.009704,-0.001588,-0.001782,...,-0.004767,-0.000139,0.004343,-0.000845,0.001064,0.003169,0.003155,0.000911,0.001869,-0.002004
4,0.002243,0.001546,0.004814,0.001409,0.009516,-0.003293,-0.003278,-0.009673,-0.001581,-0.001775,...,-0.004226,-0.000133,0.003845,-0.000748,0.001012,0.002790,0.002691,0.000864,0.001816,-0.001892
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20101,0.002243,-0.000946,-0.002601,0.001704,0.001420,0.003763,-0.005474,0.002576,-0.002881,-0.005222,...,-0.000147,0.000379,0.000823,-0.000758,0.001046,-0.001314,-0.000312,0.000773,0.000136,0.000108
20102,0.002243,-0.000957,-0.002647,0.001759,0.001477,0.003950,-0.005782,0.002744,-0.003108,-0.005686,...,-0.000653,0.001739,0.003981,-0.003748,0.005424,-0.006877,-0.001718,0.004472,0.000818,0.000677
20103,0.002243,-0.000956,-0.002642,0.001754,0.001472,0.003933,-0.005753,0.002729,-0.003086,-0.005642,...,-0.000596,0.001586,0.003624,-0.003409,0.004927,-0.006245,-0.001557,0.004049,0.000740,0.000612
20104,0.002243,-0.000957,-0.002647,0.001759,0.001478,0.003953,-0.005785,0.002746,-0.003110,-0.005691,...,-0.000660,0.001759,0.004027,-0.003792,0.005489,-0.006959,-0.001739,0.004527,0.000828,0.000686


## Heat kernel signature

In [6]:
heat_kernel_signature = nso.spectral.heat_kernel_signature(viewer.layers[0].data)
feature_table = pd.DataFrame(
    heat_kernel_signature, columns=[f"HKS_eigenvector_{i}" for i in range(10)]
)
viewer.layers[0].features = feature_table

In [7]:
data = list(viewer.layers[0].data)
data[2] = feature_table["HKS_eigenvector_4"].values
viewer.layers[0].data = tuple(data)